In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/legal-dataset/legal_docs_modified.csv


In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
df=pd.read_csv("/kaggle/input/legal-dataset/legal_docs_modified.csv")

In [4]:
df.head()

,Unnamed: 0,clause_text,clause_type,totalwords,totalletters,clause_status
0,0,"Make any Investments, except:",investments,4.0,30.0,0
1,1,No more than 45% of the “value” (as defined i...,investments,76.0,460.0,0
2,2,"Make or hold any Investments, except:",investments,6.0,38.0,0
3,3,The SubAdviser is hereby authorized and direc...,investments,228.0,1474.0,1
4,4,"Make any advance, loan, extension of credit (...",investments,52.0,329.0,0


In [5]:
df.columns

Index(['Unnamed: 0', 'clause_text', 'clause_type', 'totalwords',
       'totalletters', 'clause_status'],
      dtype='object')

In [7]:
df.isnull().sum()

Unnamed: 0        0
clause_text      43
clause_type       0
totalwords       26
totalletters     26
clause_status     0
dtype: int64

In [10]:
df = df.dropna(subset=['clause_text'])

df['totalwords'] = df['totalwords'].fillna(
    df['clause_text'].apply(lambda x: len(str(x).split()))
)

df['totalletters'] = df['totalletters'].fillna(
    df['clause_text'].apply(lambda x: sum(c.isalpha() for c in str(x))))

In [11]:
df.isnull().sum()

Unnamed: 0       0
clause_text      0
clause_type      0
totalwords       0
totalletters     0
clause_status    0
dtype: int64

In [12]:
df.duplicated().sum()

np.int64(0)

In [15]:
df["clause_status"].value_counts()

clause_status
1    12816
0     8328
Name: count, dtype: int64

In [19]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text_nltk(text):

    text = str(text).lower()
    
    text = re.sub(r'[^a-z\s]', '', text)
    
    words = word_tokenize(text)
    
    filtered_words = [word for word in words if word not in stop_words]
    
    return " ".join(filtered_words)


df['cleaned_clause'] = df['clause_text'].apply(preprocess_text_nltk)


df[['clause_text', 'cleaned_clause']].head()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,clause_text,cleaned_clause
0,"Make any Investments, except:",make investments except
1,No more than 45% of the “value” (as defined i...,value defined section investment company act a...
2,"Make or hold any Investments, except:",make hold investments except
3,The SubAdviser is hereby authorized and direc...,subadviser hereby authorized directed hereby a...
4,"Make any advance, loan, extension of credit (...",make advance loan extension credit way guarant...


In [21]:
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_columns', None)
df[['clause_text', 'cleaned_clause']].head()

,clause_text,cleaned_clause
0,"Make any Investments, except:",make investments except
1,"No more than 45% of the “value” (as defined in Section 2(a)(41) of the Investment Company Act of 1940, as amended (“Investment Company Act”)) of the Company’s total assets consist of, and no more than 45% of the Company’s net income after taxes is derived from, securities other than “Government Securities” (as defined in Section 2(a)(16) of the Investment Company Act) or money market funds meeting the conditions of Rule 2a-7 of the Investment Company Act.",value defined section investment company act amended investment company act companys total assets consist companys net income taxes derived securities government securities defined section investment company act money market funds meeting conditions rule investment company act
2,"Make or hold any Investments, except:",make hold investments except
3,"The SubAdviser is hereby authorized and directed and hereby agrees, subject to the stated investment policies and restrictions of the Fund as set forth in the Fund's prospectus, summary prospectus and/or statement of additional information, in each case as currently in effect and as supplemented or amended from time to time (collectively referred to hereinafter as the “Prospectus”) and subject to the directions and continuous oversight of the Adviser and the Fund’s Board of Trustees, to (i) regularly provide investment advice and recommendations to the Fund with respect to the Fund’s investments, investment policies and the purchase and sale of securities and other investments; (ii) supervise continuously the investment program of the Fund and the composition of its portfolio and determine what securities and other investments shall be purchased or sold by the Fund; and (iii) arrange, subject to the provisions of paragraphs (c) and (d) below, for the purchase of securities, and other investments for the Fund and the sale of securities and other investments held in the portfolio of the Fund. The Adviser agrees to provide the SubAdviser with such assistance as may be reasonably requested by the SubAdviser in connection with its activities under this Agreement, including, without limitation, information concerning the Fund, its assets available, or to become available, for investment and generally as to the conditions of the Fund or the Fund’s affairs.",subadviser hereby authorized directed hereby agrees subject stated investment policies restrictions fund set forth funds prospectus summary prospectus andor statement additional information case currently effect supplemented amended time time collectively referred hereinafter prospectus subject directions continuous oversight adviser funds board trustees regularly provide investment advice recommendations fund respect funds investments investment policies purchase sale securities investments ii supervise continuously investment program fund composition portfolio determine securities investments shall purchased sold fund iii arrange subject provisions paragraphs c purchase securities investments fund sale securities investments held portfolio fund adviser agrees provide subadviser assistance may reasonably requested subadviser connection activities agreement including without limitation information concerning fund assets available become available investment generally conditions fund funds affairs
4,"Make any advance, loan, extension of credit (by way of guaranty or otherwise) or capital contribution to, or purchase any Capital Stock, bonds, notes, debentures or other debt securities of, or any assets constituting a business unit of, or make any other investment in, any Person (all of the foregoing, “Investments”), except:",make advance loan extension credit way guaranty otherwise capital contribution purchase capital stock bonds notes debentures debt securities assets constituting business unit make investment person foregoing investments except


In [20]:
df.iloc[1]

Unnamed: 0                                                        1
clause_text        No more than 45% of the “value” (as defined i...
clause_type                                             investments
totalwords                                                     76.0
totalletters                                                  460.0
clause_status                                                     0
cleaned_clause    value defined section investment company act a...
Name: 1, dtype: object

In [28]:


stop_words = set(stopwords.words('english'))

# The ultimate legal rescue list
words_to_keep = {
    'no', 'not', 'nor', 'except', 'against', 'without', 'only', 'any', 'but',

    'more', 'than', 'less', 'least', 'greater', 'equal', 'over', 'under', 'above', 'below',
    
    'if', 'until', 'while',
    
    'all', 'both', 'each', 'other', 'some', 'such',
    
    'prior', 'after', 'before', 'during', 'once',

    'can', 'will', 'should',

    'a'
}

legal_stop_words = stop_words - words_to_keep

def preprocess_legal_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s%$]', ' ', text)
    
    words = word_tokenize(text)
    
    filtered_words = [word for word in words if word not in legal_stop_words]
    
    clean_text = " ".join(filtered_words)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    return clean_text


df['cleaned_clause'] = df['clause_text'].apply(preprocess_legal_text)

In [29]:
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_columns', None)
df[['clause_text', 'cleaned_clause']].head()

,clause_text,cleaned_clause
0,"Make any Investments, except:",make any investments except
1,"No more than 45% of the “value” (as defined in Section 2(a)(41) of the Investment Company Act of 1940, as amended (“Investment Company Act”)) of the Company’s total assets consist of, and no more than 45% of the Company’s net income after taxes is derived from, securities other than “Government Securities” (as defined in Section 2(a)(16) of the Investment Company Act) or money market funds meeting the conditions of Rule 2a-7 of the Investment Company Act.",no more than 45 % value defined section 2 a 41 investment company act 1940 amended investment company act company total assets consist no more than 45 % company net income after taxes derived securities other than government securities defined section 2 a 16 investment company act money market funds meeting conditions rule 2a 7 investment company act
2,"Make or hold any Investments, except:",make hold any investments except
3,"The SubAdviser is hereby authorized and directed and hereby agrees, subject to the stated investment policies and restrictions of the Fund as set forth in the Fund's prospectus, summary prospectus and/or statement of additional information, in each case as currently in effect and as supplemented or amended from time to time (collectively referred to hereinafter as the “Prospectus”) and subject to the directions and continuous oversight of the Adviser and the Fund’s Board of Trustees, to (i) regularly provide investment advice and recommendations to the Fund with respect to the Fund’s investments, investment policies and the purchase and sale of securities and other investments; (ii) supervise continuously the investment program of the Fund and the composition of its portfolio and determine what securities and other investments shall be purchased or sold by the Fund; and (iii) arrange, subject to the provisions of paragraphs (c) and (d) below, for the purchase of securities, and other investments for the Fund and the sale of securities and other investments held in the portfolio of the Fund. The Adviser agrees to provide the SubAdviser with such assistance as may be reasonably requested by the SubAdviser in connection with its activities under this Agreement, including, without limitation, information concerning the Fund, its assets available, or to become available, for investment and generally as to the conditions of the Fund or the Fund’s affairs.",subadviser hereby authorized directed hereby agrees subject stated investment policies restrictions fund set forth fund prospectus summary prospectus statement additional information each case currently effect supplemented amended time time collectively referred hereinafter prospectus subject directions continuous oversight adviser fund board trustees regularly provide investment advice recommendations fund respect fund investments investment policies purchase sale securities other investments ii supervise continuously investment program fund composition portfolio determine securities other investments shall purchased sold fund iii arrange subject provisions paragraphs c below purchase securities other investments fund sale securities other investments held portfolio fund adviser agrees provide subadviser such assistance may reasonably requested subadviser connection activities under agreement including without limitation information concerning fund assets available become available investment generally conditions fund fund affairs
4,"Make any advance, loan, extension of credit (by way of guaranty or otherwise) or capital contribution to, or purchase any Capital Stock, bonds, notes, debentures or other debt securities of, or any assets constituting a business unit of, or make any other investment in, any Person (all of the foregoing, “Investments”), except:",make any advance loan extension credit way guaranty otherwise capital contribution purchase any capital stock bonds notes debentures other debt securities any 

In [ ]:
# --- WHY WE DITCHED BASIC NLTK & TF-IDF ---
# We tried standard text cleaning first, but it literally deleted words like "no" and "under".
# It turned strict legal limits like "no more than 45%" into just "45%", ruining the meaning.
# It basically blended our contracts into alphabet soup, so we had to throw it out.

# --- WHY N-GRAMS WEREN'T ENOUGH ---
# We tried grouping words with N-Grams so it would read "no more than" as one big chunk.
# But it's just dumb pattern memorization; it doesn't actually understand the English language.
# If a new contract said "not exceeding" instead, the model would completely panic and fail.

# --- WHY WORD2VEC WAS A HARD PASS ---
# Word2Vec is cool, but to get a full sentence score you have to average all the word numbers together.
# That means "company sues employee" becomes the exact same math as "employee sues company".
# In legal docs, word order and who does what kinda matters a lot, so this was a terrible idea.

In [30]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib


# 'all-MiniLM-L6-v2' lightweight,fast, high-quality sentence embeddings.
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


X = embedding_model.encode(df['clause_text'].astype(str).tolist(), show_progress_bar=True)

y = df['clause_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


classifier = LogisticRegression(class_weight='balanced', max_iter=1000)
classifier.fit(X_train, y_train)


y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))


joblib.dump(classifier, 'legal_risk_classifier.pkl')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/661 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1666
           1       0.92      0.89      0.90      2563

    accuracy                           0.89      4229
   macro avg       0.88      0.89      0.88      4229
weighted avg       0.89      0.89      0.89      4229



['legal_risk_classifier.pkl']

In [34]:
# 1. Write a brand new clause the model has NEVER seen
my_fake_clause = "The employee shall assume unlimited liability for all damages, waives all rights to legal counsel, and must pay a $500,000 penalty for any breach of this agreement without notice."

# 2. Turn your custom sentence into numbers (GPS coordinates) using the embedding model
my_custom_embedding = embedding_model.encode([my_fake_clause])

# 3. Ask the trained classifier which side of the line it lands on!
prediction = classifier.predict(my_custom_embedding)

if prediction[0] == 1:
    print("RISKY")
else:
    print("NORMAL/SAFE")

🚨 THE MODEL SAYS THIS IS RISKY!


In [35]:

df.to_csv('legal_docs_cleaned.csv', index=False)

